## Build submit script and XML file

In [1]:
import os
import subprocess as sp
import argparse
import shutil
import yaml

In [15]:

def f_write_config(input_dict,fname):
    
    with open(fname,'w') as f:
        yaml.dump(input_dict,f,sort_keys=False)
    
def f_read_config(fname):
    
    with open(fname,'r') as f:
        config_dict=yaml.load(f, Loader=yaml.SafeLoader)
        
    return config_dict

In [3]:
def f_build_submit_script(dict_pars,fname):
    
    submit_strg='''#!/bin/bash
    ################    

    #SBATCH -N 1
    #SBATCH -n 8
    #SBATCH --exclusive
    #SBATCH --gpu-bind=map_gpu:0,1,2,3,7,6,5,4
    #SBATCH -c 8
    # #SBATCH -p debug

    Lattice="{Lx}.{Lx}.{Lx}.{Lt}"
    MPIGrid="{mpi}"

    machine=tioga
    output_dir=${{machine}}-${{SLURM_JOB_ID}}
    mkdir $output_dir

    cd $SLURM_SUBMIT_DIR
    exec > ${{SLURM_SUBMIT_DIR}}/$output_dir/${{SLURM_JOB_NAME}}-${{SLURM_JOB_ID}}.${{machine}}.out 2>&1


    echo "--------------------------------"
    echo "SLURM job running on: `hostname`"
    echo "in directory:         `pwd`"
    echo "SLURM jobid:          ${{SLURM_JOB_ID}}"
    echo "SLURM #nodes:         ${{SLURM_NNODES}}"
    echo "SLURM tasks per node: ${{SLURM_TASKS_PER_NODE}}"
    echo "SLURM #ntasks:        ${{SLURM_NTASKS}}" 
    echo "Nodefile: ${{SLURM_JOB_NODELIST}}"
    echo "--------------------------------"
    lscpu

    echo "# time-start "`date`
    TotalTic=`date +%s`

    export BIND="--cpu-bind=verbose,map_ldom:3,3,1,1,2,2,0,0"

    export MPIR_CVAR_GPU_EAGER_DEVICE_MEM=16384
    export MPICH_GPU_SUPPORT_ENABLED=1
    export MPICH_SMP_SINGLE_COPY_MODE=CMA
    export OMP_NUM_THREADS=8
    export MPICH_OFI_NIC_POLICY=GPU

    source $GRID_DIR/setup_env.sh
    module list

    cat << EOF > $output_dir/select_gpu
    #!/bin/bash

    export ROCR_VISIBLE_DEVICES=\$SLURM_LOCALID
    exec \$*
    EOF
    chmod +x ./$output_dir/select_gpu

    input=DWFtest
    sed -e "s/MACHINE_JOBID_/$output_dir/" < $input.xml > $output_dir/${{input}}.xml

    ## Need global variable
    BINARY=$GRID_DIR/grid_prefix/bin/HadronsXmlRun

    PARAMS=" --accelerator-threads 8 --grid ${{Lattice}} --mpi ${{MPIGrid}} --comms-sequential --shm 2048 --shm-mpi 1"

    echo "srun --gpus-per-task 1 -n{nprocs} $BIND ./$output_dir/select_gpu $BINARY $output_dir/${{input}}.xml $PARAMS"
    srun --gpus-per-task 1 -n{nprocs} $BIND ./$output_dir/select_gpu $BINARY $output_dir/${{input}}.xml  $PARAMS

    ############################################################################################# 
    ############################################################################################# 

    TotalToc=`date +%s`
    echo "# time-finish "`date`

    TotalTime=$(( $TotalToc - $TotalTic ))
    TotalHours=`echo "$TotalTime / 3600" | bc -l`
    echo "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
    echo "Total time  $TotalTime [sec] = $TotalHours [h]"
    echo "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
'''.format(**dict_pars)
    
    
    with open(fname,'w') as f: f.write(submit_strg)
        
    return submit_strg



In [59]:
def f_build_xml(dict_pars,fname):
    xml_strg='''<?xml version="1.0"?>
<grid>
  <parameters>
    <trajCounter>
      <start>{start}</start>
      <end>{end}</end>
      <step>{step}</step>
    </trajCounter>
    <genetic>
      <popSize>20</popSize>
      <maxGen>1000</maxGen>
      <maxCstGen>100</maxCstGen>
      <mutationRate>0.1</mutationRate>
    </genetic>
    <runId>test</runId>
    <graphFile></graphFile>
    <scheduleFile></scheduleFile>
    <saveSchedule>false</saveSchedule>
    <parallelWriteMaxRetry>-1</parallelWriteMaxRetry>
  </parameters>
  <modules>
    <module>
      <id>
        <name>gauge</name>
        <type>MIO::LoadNersc</type>
      </id>
      <options>
        <file>./{file_strg}</file>
      </options>
    </module>
<!--           Sources/Sinks             -->  
    <module>
      <id>
        <name>pt</name>
        <type>MSource::Point</type>
      </id>
      <options>
        <position>0 0 0 0</position>
      </options>
    </module>
    <module>
      <id>
        <name>sink_000</name>
        <type>MSink::ScalarPoint</type>
      </id>
      <options>
        <mom>0 0 0 0</mom>
      </options>
    </module>
<!-- ################################### -->
<!-- ###### Begin s Quark Modules ###### -->
<!-- ################################### -->    
<!-- Action for Strange Quark -->
    <module>
      <id>
        <name>DWF_m</name>
        <type>MAction::MobiusDWF</type>
      </id>
      <options>
        <gauge>gauge</gauge>
        <Ls>{dwf_Ls}</Ls>
        <mass>{m_s}</mass>
        <M5>{dw_M5}</M5>
        <b>{dw_b}</b>
        <c>{dw_c}</c>
        <boundary>1 1 1 -1</boundary>
        <twist>0. 0. 0. 0.</twist>
      </options>
    </module>
<!-- Solver for Strange Quark Action-->
    <module>
      <id>
        <name>CG_s</name>
        <type>MSolver::RBPrecCG</type>
      </id>
      <options>
        <action>DWF_m</action>
        <maxIteration>8000</maxIteration>
        <residual>1e-09</residual>
        <eigenPack></eigenPack>
      </options>
    </module>
<!-- Strange Quark Propagator-->
    <module>
      <id>
        <name>Strange_Quark</name>
        <type>MFermion::GaugeProp</type>
      </id>
      <options>
        <source>pt</source>
        <solver>CG_s</solver>
      </options>
    </module>
<!-- -->
   <module>
      <id>
        <name>mres_Strange_Quark</name>
        <type>MContraction::WardIdentity</type>
      </id>
      <options>
        <prop>Strange_Quark_5d</prop>
        <action>DWF_m</action>
        <mass>0.32</mass>
    <source>pt</source>
        <test_axial>true</test_axial>
        <output>./MACHINE_JOBID_/prop_gauge_DWF</output>
      </options>
    </module>
<!-- ######################################### -->
<!-- #### Begin Meson Contraction Modules #### -->
<!-- ######################################### -->
<!--       Light-Light Contraction       -->
<!--        (Eta_s 2-pt function)        -->
    <module>
      <id>
        <name>meson_pt_ll</name>
        <type>MContraction::Meson</type>
      </id>
      <options>
        <q1>Strange_Quark</q1>
        <q2>Strange_Quark</q2>
        <gammas>(Gamma5 Gamma5)</gammas>
        <sink>sink_000</sink>
        <output>./MACHINE_JOBID_/eta_s_2pt</output>
      </options>
    </module>
  </modules>
</grid>
    '''.format(**dict_pars)
        
        
    with open(fname,'w') as f: f.write(xml_strg)

    return xml_strg


- Read folder with HMC run
- Read 'config.yaml' 
- Extract outer dirname
- Create mesure directory
- Update dict_pars and write 'config_meas.yaml'
- Create Submit script
- Create XML script

In [40]:
hmc_dir='/usr/WS2/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02'
input_dict=f_read_config(hmc_dir+'/config.yaml')


In [41]:

start,end,step=90,210,10
N=1
mx,my,mz,mt=2,2,2,1

dict_pars={
## Can remove this after saving in config.yaml
'm_s':0.32,'dw_M5':1.8,'dw_b':1.5,'dw_c':0.5,

'mpi':".".join([str(i) for i in [mx,my,mz,mt]]),
'nprocs': mx*my*mz*mt,

'file_strg':hmc_dir+'/ckpoint_lat',
'start':start, 'end':end, 'step':10
}
    

## Copy keys from input_dict
for key in input_dict.keys():
    print(key)
    dict_pars[key]=input_dict[key]


Lx
Lt
F_action
traj_l
md_steps
beta
mf
dwf_Ls


In [42]:
dict_pars

{'m_s': 0.32,
 'dw_M5': 1.8,
 'dw_b': 1.5,
 'dw_c': 0.5,
 'mpi': '2.2.2.1',
 'nprocs': 8,
 'file_strg': '/usr/WS2/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/phase_diagram_mobius_Ls8/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02/ckpoint_lat',
 'start': 90,
 'end': 210,
 'step': 10,
 'Lx': 16,
 'Lt': 8,
 'F_action': 'Mobius_dwf',
 'traj_l': 1,
 'md_steps': 10,
 'beta': 6.0,
 'mf': 0.02,
 'dwf_Ls': 8}

In [76]:
dest='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/'

assert os.path.exists(dest), "Top directory %s doesn't exist"%(dest)

## Top folder name
top_fldr=hmc_dir.split('/')[-1]
fldr=dest+top_fldr
print(fldr)

## Ensure you don't overwrite existing run
count=1
while os.path.exists(fldr): 
    count+=1
    print("Directory %s exists",fldr)
    fldr=fldr+'_%s'%(count)
    print("Creating modified directory",fldr)
    
    if count>5:    
        print("Too many created directories")
        break
    

os.makedirs(fldr)

## Write config file
fname=fldr+'/meas_config.yaml'
f_write_config(dict_pars,fname)

## Write submit script
submit_strg=f_build_submit_script(dict_pars,fldr+'/test_submit_write.sh')

## Write XML file
xml_strg=f_build_xml(dict_pars,fldr+'/test_xml_write.sh')
    
    

/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02
Directory %s exists /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02
Creating modified directory /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02_2
Directory %s exists /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02_2
Creating modified directory /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02_2_3
Directory %s exists /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_june15_mobius/dm_tests/meas_mobius/run3_/run_Lx-16_Lt-8_Ls-8_beta-6.0_mf-0.02_2_3
Creating modified directory /usr/workspace/lsd/ayyar1/pr